# Simple Text summarization- using unsupervised learning

###  Given any Input document → sentences similarity → weight sentences → select sentences with higher rank.
#### Input article → split into sentences → remove stop words → build a similarity matrix → generate rank based on matrix → pick top N sentences for summary.


In [1]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

In [2]:
def read_file(file_name):
    file = open(file_name, "r")
    return nltk.tokenize.sent_tokenize(file.read())

###  similarity matrix

In [3]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix

### Generating a summary

In [21]:
def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_file(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

    # print("Indexes of top ranked_sentence order are ", ranked_sentence)
    print("# original sentences" , len(sentences))
    print("# summarized sentences" , (int)(len(sentences)/2)+1)
    for i in range((int)(len(sentences)/2)+1):
        summarize_text.append("".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("Summarized Text: \n", ". ".join(summarize_text))

# let's begin
generate_summary( "cloud_computing_eurekalert.txt", 9)

# original sentences 22
# summarized sentences 12
Summarized Text: 
 The Galactic RainCloudS project, an initiative led by members of the Faculty of Physics, the Institute of Cosmos Sciences (ICCUB) of the University of Barcelona and the Institute for Space Studies of Catalonia (IEEC), was awarded the first position in the framework of the Cloud Funding for Research call of the European project Open Clouds For Research Environments (OCRE).. Enrique González Lezana, head of cloud sales specialist at Telefónica Tech, says that “Telefónica has accompanied the University of Barcelona in the definition and unfolding of the Google Cloud architecture, where the required hypercomputing solution to work on the Galactic RainCloudS project will be hosted”.. “Cloud computing is like renting powerful customized computers, for a certain period of time, which will enable us to make the necessary calculations to study the interaction between galaxies”, notes Mercè Romero, researcher at ICCUB.. The res